In [ ]:
pip install gradio

In [17]:


import gradio as gr
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
import io
from PIL import Image


def segment_sky_with_kmeans(image_rgb):
    h, w = image_rgb.shape[:2]
    reshaped_img = image_rgb.reshape((-1, 3)).astype(np.float32)

      # Using 2 cluster centers
    k = 2
    _, labels, centers = cv2.kmeans(reshaped_img, k, None, (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 0.2), 10, cv2.KMEANS_RANDOM_CENTERS)

    # Assuming the sky cluster has higher brightness
    sky_cluster_index = np.argmax(np.mean(centers, axis=1))
    sky_mask = (labels.reshape((h, w)) == sky_cluster_index)

    return sky_mask.astype(np.uint8) * 255





In [18]:
def process_image_and_display_all_steps(image_rgb):


    sky_mask_kmeans = segment_sky_with_kmeans(image_rgb)

    image_hsv = cv2.cvtColor(image_rgb, cv2.COLOR_RGB2HSV)

    # Refine the blue sky color thresholds
    lower_blue = np.array([70, 50, 100])  # Increase the lower bounds
    upper_blue = np.array([130, 255, 255])  # Adjust the hue to be more selective

    mask_blue = cv2.inRange(image_hsv, lower_blue, upper_blue)

    # Keep the sunset sky color thresholds unchanged
    lower_orange_red = np.array([0, 100, 100])
    upper_orange_red = np.array([30, 255, 255])
    mask_orange_red = cv2.inRange(image_hsv, lower_orange_red, upper_orange_red)

    # Combine the blue and orange/red sky masks
    sky_mask = cv2.bitwise_or(mask_blue, mask_orange_red)

    # Only apply the mask to the upper part of the image
    h, w = sky_mask.shape
    two_thirds_mask = np.zeros_like(sky_mask)
    two_thirds_mask[:2*h//3, :] = 1  # Create a mask for the upper two-thirds
    sky_mask_two_thirds = cv2.bitwise_and(sky_mask, sky_mask, mask=two_thirds_mask)

    # Clean up the mask with morphological operations
    kernel = np.ones((7, 7), np.uint8)
    mask_morph = cv2.morphologyEx(sky_mask, cv2.MORPH_CLOSE, kernel)
    mask_morph = cv2.morphologyEx(mask_morph, cv2.MORPH_OPEN, kernel)

    # Fill the sky area with white color
    image_with_white_sky = image_rgb.copy()
    image_with_white_sky[mask_morph != 0] = (255, 255, 255)

    # Display the key steps of the process
    fig, axs = plt.subplots(1, 6, figsize=(18, 5))

    axs[0].imshow(image_rgb)
    axs[0].set_title('Original Image')
    axs[0].axis('off')

    axs[1].imshow(sky_mask_kmeans, cmap='gray')
    axs[1].set_title('K-means Sky Mask')
    axs[1].axis('off')

    axs[2].imshow(mask_blue, cmap='gray')
    axs[2].set_title('Blue Sky Mask')
    axs[2].axis('off')

    axs[3].imshow(mask_orange_red, cmap='gray')
    axs[3].set_title('Sunset Sky Mask')
    axs[3].axis('off')

    axs[4].imshow(mask_morph, cmap='gray')
    axs[4].set_title('Morphological Mask')
    axs[4].axis('off')

    axs[5].imshow(image_with_white_sky)
    axs[5].set_title('Image with White Sky')
    axs[5].axis('off')

    plt.tight_layout()


    # Save the plot to a buffer instead of showing it
    buf = io.BytesIO()
    plt.savefig(buf, format='png')
    buf.seek(0)
    plt.close()

    # Convert buffer to PIL image and return
    img = Image.open(buf)
    return img

In [19]:
def process_image_and_display_all_steps_gradio(input_image):
    img = process_image_and_display_all_steps(input_image)
    return img



In [20]:
demo = gr.Interface(
    fn=process_image_and_display_all_steps_gradio,
    inputs="image",
    outputs="image"
)
demo.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://8715c0ad40de83342b.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
